# Import packages

In [1]:
import pandas as pd
import json
import random
import os

# Import data

In [2]:
# Load the JSON file
with open("questions.json", "r", encoding="utf-8-sig") as f:
    data = json.load(f)

print(type(data))
print(data[0])

<class 'list'>
{'task_id': '8e867cd7-cff9-4e6c-867a-ff5ddc2550be', 'question': 'How many studio albums were published by Mercedes Sosa between 2000 and 2009 (included)? You can use the latest 2022 version of english wikipedia.', 'Level': '1', 'file_name': ''}


In [3]:
# Select random question
random_question = random.choice(data)
print(random_question)

{'task_id': 'cabe07ed-9eca-40ea-8ead-410ef5e83f91', 'question': "What is the surname of the equine veterinarian mentioned in 1.E Exercises from the chemistry materials licensed by Marisa Alviar-Agnew & Henry Agnew under the CK-12 license in LibreText's Introductory Chemistry materials as compiled 08/21/2023?", 'Level': '1', 'file_name': ''}


# Set up LLM with Ollama 
I wanted to use LlamaIndex's HuggingFaceAPI but my pc configurations won't let me use this API

In [6]:
# before running the following code I first installed Ollama () on my pc
# then in the terminal  I tan "ollama pull qwen3"

from llama_index.llms.ollama import Ollama

llm = Ollama(
    model="qwen3:0.6b", #qwen3:latest
    #request_timeout=120.0,
    # Manually set the context window to limit memory usage
    context_window=8000,
)

In [7]:
# test
response = llm.complete('What\'s the capital of France?')

In [8]:
print(response)

<think>
Okay, the user is asking about the capital of France. I remember that France is a country in Europe, and the capital is Paris. But wait, let me make sure I'm not confusing it with other countries. For example, in the US, the capital is Washington, D.C., and in the UK, it's London. France's capital is definitely Paris. I should double-check some sources to confirm this. Oh, right! Paris is the capital, and it's also the capital of the European Union. No other cities are commonly referred to as the capital here. So the answer is Paris.
</think>

The capital of France is **Paris**.


# Set up Llamaindex (tools, agents, prompt)

# Run agent on question to test it